# SQL

# Задачи

- Исследуйте таблицы — выведите первые строки
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
- Общий вывод

# Цель

- Проанализировать базу данных. Получить информацию о книгах, издательствах, авторах и пользовательских обзорах книг. Сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
books = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(books), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
def select(sql):
    return pd.read_sql(sql = text(sql),con = engine)

## Исследуйте таблицы — выведите первые строки

In [3]:
authors = '''SELECT * FROM authors LIMIT 5'''
select(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
publishers = '''SELECT * FROM publishers LIMIT 5'''
select(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
ratings = '''SELECT * FROM ratings LIMIT 5'''
select(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
reviews = '''SELECT * FROM reviews LIMIT 5'''
select(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитайте, сколько книг вышло после 1 января 2000 года;

In [7]:
count = '''SELECT COUNT(title) FROM books WHERE publication_date > '2000-01-01' '''
select(count)

,count
0,819


### Вывод: 819 книг было выпущено после 01.01.2000 года

## Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [8]:
count_rev_rating = '''
SELECT 
title,
COUNT(DISTINCT(review_id)),
ROUND(avg(rating),2)

FROM books AS b 

LEFT JOIN ratings AS rat 
ON b.book_id = rat.book_id

LEFT JOIN reviews AS rev 
ON b.book_id = rev.book_id

GROUP BY b.book_id

ORDER BY round DESC, count DESC 
'''
select(count_rev_rating)

,title,count,round
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


### Вывод: максимальный средний рейтинг и количество обзоров у книги A Dirty Job (Grim Reaper #1), минимальный средний рейтинг и количество обзоров у книги "Harvesting the Heart"

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
publishers_50 = '''
SELECT
publisher,
COUNT(book_id)

FROM publishers as p

JOIN books AS b ON b.publisher_id = p.publisher_id

WHERE b.num_pages > 50

GROUP BY publisher

ORDER BY count DESC

LIMIT 3 '''
select(publishers_50)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


### Вывод: Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books, количество книг составляет 42 шт. Далее идет издательство Vintage - 31 книга. Далее Grand Central Publishing - 25 книг.

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
author_50 = '''
SELECT author,
ROUND(AVG(rating),3)

FROM authors AS a

JOIN books AS b ON a.author_id = b.author_id

JOIN ratings AS r ON b.book_id = r.book_id

WHERE b.book_id IN (
SELECT book_id
FROM ratings 
GROUP BY book_id
HAVING COUNT(rating_id) >= 50
ORDER BY COUNT(rating_id) DESC)

GROUP BY author

ORDER BY round DESC

LIMIT 3
'''
select(author_50)

,author,round
0,J.K. Rowling/Mary GrandPré,4.287
1,Markus Zusak/Cao Xuân Việt Khương,4.264
2,J.R.R. Tolkien,4.247


### Вывод: С самым высоким средним рейтингом автор J.K. Rowling/Mary GrandPré - рейтинг 4.28. Далее Markus Zusak/Cao Xuân Việt Khương - рейтинг 4.26. Далее J.R.R. Tolkien	- рейтинг 4.24 (учитывались только книги с 50 и более оценками)

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
reviews_48 = '''
SELECT ROUND(avg(count)) 

FROM (SELECT COUNT(text)
FROM reviews AS r
FULL OUTER JOIN (SELECT username, COUNT(rating_id)
FROM ratings
GROUP BY username) AS cr ON cr.username = r.username
WHERE count > 48
GROUP BY r.username) as total_count
'''
select(reviews_48)

,round
0,24.0


### Вывод: Среднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет 24.

## Общий вывод:

- 819 книг было выпущено после 01.01.2000 года;
- Максимальный средний рейтинг и количество обзоров у книги A Dirty Job (Grim Reaper #1), минимальный средний рейтинг и количество обзоров у книги "Harvesting the Heart";
- Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books, количество книг составляет 42 шт. Далее идет издательство Vintage - 31 книга. Далее Grand Central Publishing - 25 книг;
- С самым высоким средним рейтингом автор J.K. Rowling/Mary GrandPré: рейтинг 4.28. Далее Markus Zusak/Cao Xuân Việt Khương: рейтинг 4.26. Далее J.R.R. Tolkien: рейтинг 4.24 (учитывались только книги с 50 и более оценками); 
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет 24.